
# data issue : The threshold score for Model3 should be 20 rather than 30. Setting is as 30 will make it never get hit.


As I don't have database on my PC, I will use python and provide a pseudo sql code. As SQL is very easy.



In [1]:
import pandas as pd
import numpy as np

# 1. Seperate three tables into Model_table, Rule_table and Hit table

In [2]:
model_table = pd.read_csv('Model_table.csv') # after typo fixed
rule_table = pd.read_csv('Rule_table.csv') 
hit_table = pd.read_csv('Hit_table.csv')

In [3]:
model_table.shape

(13, 2)

In [4]:
model_table.head()

,Model_name,Threshold
0,Mod1_INV,50
1,Mod1_CP,30
2,Mod2_CP,50
3,Mod2_INV,50
4,Mod3_INV,20


In [5]:
rule_table.shape

(28, 4)

In [6]:
rule_table.head()

,Model name,Rule name,Score,Customer type
0,Mod1,R01,20,Individual
1,Mod1,R02,30,Individual
2,Mod1,R03,10,Individual
3,Mod1,R04,20,Individual
4,Mod1,R01,10,Corporate


In [7]:
hit_table.shape

(5424, 4)

In [8]:
hit_table.head()

,Rule_name,Customer_type,Customer_ID,Hit_Date
0,R13,Corporate,66,2017/1/1
1,R11,Corporate,66,2017/1/2
2,R13,Corporate,66,2017/1/2
3,R13,Corporate,86,2017/1/2
4,R02,Individual,8,2017/1/3


## pseudo sql to load data into database

Create a empty table then load the corresponding csv into the table. For different database the code might be different. For example, 
using snowflake on AWS, the code is "copy into ..."

At Discover, we deal data with hundreds of millions records data. If your data is small, you can you "insert into"

In [9]:
hit_table['Hit_Date'] = pd.to_datetime(hit_table['Hit_Date'])

In [10]:
hit_table['week_sunday'] = hit_table['Hit_Date'].apply(lambda d: d + pd.Timedelta(days=(6-d.weekday())))

## 4)  If within 7 days of the monitoring period a hit on a rule for one customer is detected more than once,    only the first hit contributes to the total score.

In [11]:


hit_table['multi_hit_same_rule'] = hit_table.groupby(['Rule_name', 'Customer_ID', 'week_sunday'])['Hit_Date'].rank(method='first').astype(int)

In [12]:
hit_table_dedup = hit_table.loc[hit_table['multi_hit_same_rule'] == 1]

sql:

select  a.*  
from (  
    select Rule_name, Customer_ID, week_sunday, Hit_Date,  
    row_number() over( partition by Rule_name, Customer_ID, week_sunday order by Hit_date) as rnk  
    from hit_table  
)a  
where rnk=1;  

In [13]:
hit_table_score = hit_table_dedup.merge(rule_table, left_on=['Rule_name', 'Customer_type'], right_on=['Rule name', 'Customer type'], how='left')

In [14]:
hit_table_score

,Rule_name,Customer_type,Customer_ID,Hit_Date,week_sunday,multi_hit_same_rule,Model name,Rule name,Score,Customer type
0,R13,Corporate,66,2017-01-01,2017-01-01,1,Mod6,R13,20,Corporate
1,R11,Corporate,66,2017-01-02,2017-01-08,1,Mod5,R11,10,Corporate
2,R13,Corporate,66,2017-01-02,2017-01-08,1,Mod6,R13,20,Corporate
3,R13,Corporate,86,2017-01-02,2017-01-08,1,Mod6,R13,20,Corporate
4,R02,Individual,8,2017-01-03,2017-01-08,1,Mod1,R02,30,Individual
...,...,...,...,...,...,...,...,...,...,...
2318,R12,Corporate,80,2017-06-30,2017-07-02,1,Mod5,R12,5,Corporate
2319,R12,Corporate,86,2017-06-30,2017-07-02,1,Mod5,R12,5,Corporate
2320,R12,Corporate,92,2017-06-30,2017-07-02,1,Mod5,R12,5,Corporate
2321,R12,Corporate,97,2017-06-30,2017-07-02,1,Mod5,R12,5,Corporate


In [15]:
hit_table_score_agg = hit_table_score.groupby(['Customer_ID','Customer_type', 'Model name', 'week_sunday'])['Score'].sum()

In [16]:
model_table['Model_name_shrt'] = model_table['Model_name'].str.split("_").str.get(0)
model_table['Model_name_type'] = model_table['Model_name'].str.split("_").str.get(1)
model_table['Customer_type'] = np.where(model_table['Model_name_type'] == 'INV','Individual','Corporate')


hit_table_score_agg2 = hit_table_score_agg.reset_index().merge(model_table, left_on=['Model name','Customer_type'], right_on=['Model_name_shrt', 'Customer_type'], how='left')

In [17]:
hit_table__alter = hit_table_score_agg2.loc[hit_table_score_agg2['Score'] >= hit_table_score_agg2['Threshold']]

In [18]:
from pandas.tseries.offsets import MonthEnd

hit_table__alter['mth_end'] = hit_table__alter['week_sunday'] + MonthEnd(0)

C:\Users\BO\AppData\Local\Temp\ipykernel_21388\295351293.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hit_table__alter['mth_end'] = hit_table__alter['week_sunday'] + MonthEnd(0)


In [19]:
hit_table__alter_cnt = hit_table__alter.groupby(['Model_name','mth_end'])[hit_table__alter.columns[0]].count().reset_index()

# Output 1

In [20]:
hit_table__alter_cnt.pivot(index='Model_name', columns='mth_end', values='Customer_ID').fillna(0)

mth_end,2017-01-31,2017-02-28,2017-03-31,2017-04-30,2017-05-31,2017-06-30,2017-07-31
Model_name,,,,,,,
Mod1_CP,13.0,14.0,21.0,18.0,18.0,18.0,6.0
Mod1_INV,3.0,1.0,0.0,2.0,6.0,1.0,4.0
Mod2_CP,12.0,7.0,15.0,15.0,10.0,11.0,3.0
Mod2_INV,96.0,88.0,81.0,112.0,70.0,93.0,15.0
Mod3_INV,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Mod4_INV,0.0,1.0,0.0,6.0,5.0,0.0,0.0
Mod5_CP,17.0,17.0,29.0,35.0,18.0,17.0,7.0
Mod5_INV,0.0,1.0,2.0,5.0,4.0,3.0,0.0
Mod6_CP,4.0,5.0,7.0,5.0,6.0,4.0,1.0


# Output 2

In [23]:
hit_count = hit_table.groupby('Rule_name')['Customer_ID'].count().reset_index()

In [26]:
hit_count.rename(columns={'Customer_ID':'Number of Hits'})

,Rule_name,Number of Hits
0,R01,65
1,R02,113
2,R03,302
3,R04,161
4,R05,176
5,R06,2305
6,R07,32
7,R08,78
8,R09,5
9,R10,301


In [27]:
hit_table_flag = hit_table_dedup.merge(hit_table__alter[['Customer_ID', 'Customer_type', 'Model name', 'week_sunday', 'Model_name', 'Score', 'Threshold']], how='left')

In [28]:
hit_table_flag.groupby(['Rule_name'])['Model_name'].count().reset_index().rename(columns={'Model_name':'Number of alerts'})

,Rule_name,Number of alerts
0,R01,82
1,R02,143
2,R03,387
3,R04,87
4,R05,277
5,R06,769
6,R07,20
7,R08,57
8,R09,6
9,R10,397


In [34]:
hit_table__alter.groupby(['Customer_type'])['Model_name'].count().reset_index().rename(columns={'Model_name':'Number of alerts'})

,Customer_type,Number of alerts
0,Corporate,487
1,Individual,605


In [39]:
hit_table_dedup

,Rule_name,Customer_type,Customer_ID,Hit_Date,week_sunday,multi_hit_same_rule
0,R13,Corporate,66,2017-01-01,2017-01-01,1
1,R11,Corporate,66,2017-01-02,2017-01-08,1
2,R13,Corporate,66,2017-01-02,2017-01-08,1
3,R13,Corporate,86,2017-01-02,2017-01-08,1
4,R02,Individual,8,2017-01-03,2017-01-08,1
...,...,...,...,...,...,...
5410,R12,Corporate,80,2017-06-30,2017-07-02,1
5411,R12,Corporate,86,2017-06-30,2017-07-02,1
5412,R12,Corporate,92,2017-06-30,2017-07-02,1
5413,R12,Corporate,97,2017-06-30,2017-07-02,1


In [38]:
hit_table__alter.groupby(['Model_name'])['Model_name'].count().reset_index().rename(columns={'Model_name':'Number of alerts'})

ValueError: cannot insert Model_name, already exists

In [40]:
hit_table_dedup

,Rule_name,Customer_type,Customer_ID,Hit_Date,week_sunday,multi_hit_same_rule
0,R13,Corporate,66,2017-01-01,2017-01-01,1
1,R11,Corporate,66,2017-01-02,2017-01-08,1
2,R13,Corporate,66,2017-01-02,2017-01-08,1
3,R13,Corporate,86,2017-01-02,2017-01-08,1
4,R02,Individual,8,2017-01-03,2017-01-08,1
...,...,...,...,...,...,...
5410,R12,Corporate,80,2017-06-30,2017-07-02,1
5411,R12,Corporate,86,2017-06-30,2017-07-02,1
5412,R12,Corporate,92,2017-06-30,2017-07-02,1
5413,R12,Corporate,97,2017-06-30,2017-07-02,1


In [42]:
hit_table_flag = hit_table_score.merge(hit_table__alter[['Customer_ID', 'Customer_type', 'Model name', 'week_sunday', 'Model_name', 'Score', 'Threshold']], how='left')

In [43]:
hit_table_flag

,Rule_name,Customer_type,Customer_ID,Hit_Date,week_sunday,multi_hit_same_rule,Model name,Rule name,Score,Customer type,Model_name,Threshold
0,R13,Corporate,66,2017-01-01,2017-01-01,1,Mod6,R13,20,Corporate,NaN,NaN
1,R11,Corporate,66,2017-01-02,2017-01-08,1,Mod5,R11,10,Corporate,Mod5_CP,10.0
2,R13,Corporate,66,2017-01-02,2017-01-08,1,Mod6,R13,20,Corporate,NaN,NaN
3,R13,Corporate,86,2017-01-02,2017-01-08,1,Mod6,R13,20,Corporate,NaN,NaN
4,R02,Individual,8,2017-01-03,2017-01-08,1,Mod1,R02,30,Individual,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2318,R12,Corporate,80,2017-06-30,2017-07-02,1,Mod5,R12,5,Corporate,NaN,NaN
2319,R12,Corporate,86,2017-06-30,2017-07-02,1,Mod5,R12,5,Corporate,NaN,NaN
2320,R12,Corporate,92,2017-06-30,2017-07-02,1,Mod5,R12,5,Corporate,NaN,NaN
2321,R12,Corporate,97,2017-06-30,2017-07-02,1,Mod5,R12,5,Corporate,NaN,NaN


In [44]:
hit_table_flag.groupby(['Rule_name'])['Model_name'].count().reset_index().rename(columns={'Model_name':'Number of alerts'})

,Rule_name,Number of alerts
0,R01,0
1,R02,0
2,R03,0
3,R04,0
4,R05,0
5,R06,628
6,R07,0
7,R08,0
8,R09,0
9,R10,0


In [41]:
hit_table_score

,Rule_name,Customer_type,Customer_ID,Hit_Date,week_sunday,multi_hit_same_rule,Model name,Rule name,Score,Customer type
0,R13,Corporate,66,2017-01-01,2017-01-01,1,Mod6,R13,20,Corporate
1,R11,Corporate,66,2017-01-02,2017-01-08,1,Mod5,R11,10,Corporate
2,R13,Corporate,66,2017-01-02,2017-01-08,1,Mod6,R13,20,Corporate
3,R13,Corporate,86,2017-01-02,2017-01-08,1,Mod6,R13,20,Corporate
4,R02,Individual,8,2017-01-03,2017-01-08,1,Mod1,R02,30,Individual
...,...,...,...,...,...,...,...,...,...,...
2318,R12,Corporate,80,2017-06-30,2017-07-02,1,Mod5,R12,5,Corporate
2319,R12,Corporate,86,2017-06-30,2017-07-02,1,Mod5,R12,5,Corporate
2320,R12,Corporate,92,2017-06-30,2017-07-02,1,Mod5,R12,5,Corporate
2321,R12,Corporate,97,2017-06-30,2017-07-02,1,Mod5,R12,5,Corporate


In [36]:
hit_table__alter

,Customer_ID,Customer_type,Model name,week_sunday,Score,Model_name,Threshold,Model_name_shrt,Model_name_type,mth_end
1,1,Individual,Mod1,2017-04-02,50,Mod1_INV,50,Mod1,INV,2017-04-30
3,1,Individual,Mod2,2017-01-15,50,Mod2_INV,50,Mod2,INV,2017-01-31
4,1,Individual,Mod2,2017-01-22,50,Mod2_INV,50,Mod2,INV,2017-01-31
5,1,Individual,Mod2,2017-01-29,50,Mod2_INV,50,Mod2,INV,2017-01-31
6,1,Individual,Mod2,2017-02-19,50,Mod2_INV,50,Mod2,INV,2017-02-28
...,...,...,...,...,...,...,...,...,...,...
1974,100,Corporate,Mod5,2017-07-02,15,Mod5_CP,10,Mod5,CP,2017-07-31
1991,100,Corporate,Mod7,2017-01-29,50,Mod7_CP,50,Mod7,CP,2017-01-31
1992,100,Corporate,Mod7,2017-03-05,50,Mod7_CP,50,Mod7,CP,2017-03-31
1993,100,Corporate,Mod7,2017-05-28,50,Mod7_CP,50,Mod7,CP,2017-05-31
